In [1]:
import evaluate
import json
import librosa
import pytesseract
import setuptools ## for distutils which was deprecated in python 3.12
import soundfile

import numpy as np
import tensorflow as tf

from datasets import load_dataset
from huggingface_hub import notebook_login
from pytesseract import Output

from transformers import pipeline
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import create_optimizer, AdamWeightDecay, TFAutoModelForSeq2SeqLM
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback
from transformers.pipelines.pt_utils import KeyDataset

!huggingface-cli login --token tokenizer

In [2]:
f = open('config.json')
data = json.load(f)
f.close()
tokenizer = data['write_token']

In [3]:
# notebook_login()

### [Load Dataset](https://huggingface.co/docs/transformers/tasks/summarization#load-billsum-dataset)

In [4]:
billsum = load_dataset("billsum", split="ca_test")

In [5]:
billsum = billsum.train_test_split(test_size=0.2)

### [Preprocess](https://huggingface.co/docs/transformers/tasks/summarization#preprocess)

In [6]:
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [9]:
### pytorch
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

### tensorflow
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

### [Evaluate](https://huggingface.co/docs/transformers/tasks/summarization#load-billsum-dataset)

In [10]:
rouge = evaluate.load("rouge")

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

### [Train](https://huggingface.co/docs/transformers/tasks/summarization#train)

Pytorch

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=True,
)

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_billsum["train"],
#     eval_dataset=tokenized_billsum["test"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

# trainer.train()


In [15]:
# trainer.push_to_hub()

Tensorflow

In [16]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [17]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_billsum["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_billsum["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [18]:
model.compile(optimizer=optimizer)  # No loss argument!

In [19]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

In [22]:
push_to_hub_callback = PushToHubCallback(
    output_dir="my_awesome_billsum_model",
    tokenizer=tokenizer
)

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-660b8260-56f476084913a4d745d8e9eb;22bfd734-47cf-4194-8f27-68102487d7f6)

Invalid username or password.

In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

NameError: name 'push_to_hub_callback' is not defined

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3, callbacks=callbacks)

NameError: name 'callbacks' is not defined

### [Inference](https://huggingface.co/docs/transformers/tasks/summarization#inference)

In [ ]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [ ]:
summarizer = pipeline("summarization", model="sanoosha94/my_awesome_billsum_model")
# summarizer = pipeline("summarization", model="stevhliu/my_awesome_billsum_model")
summarizer(text)

OSError: sanoosha94/my_awesome_billsum_model does not appear to have a file named config.json. Checkout 'https://huggingface.co/sanoosha94/my_awesome_billsum_model/main' for available files.